<a href="https://colab.research.google.com/github/ashinkrishnan/Next_Word_Predictor_Mal/blob/main/word_pred_mal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import heapq
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM,Activation,Dense
import pickle
from tensorflow.keras.optimizers import RMSprop



In [42]:
#sample_text = "എ.എം.ഡി. നിർമ്മിക്കുന്ന ഗ്രാഫിക്സ് പ്രോസ്സസിംഗ് യൂണിറ്റിൻറെ ബ്രാൻഡ് പേരാണ് റാഡിയോൺ R700."

In [43]:
# tokenizer = RegexpTokenizer(r'\^w+|$[0-9]+|\S+')
# words_sample = tokenizer.tokenize(sample_text)

In [119]:
#words_sample

In [10]:
#reading the dataset
path ='/content/drive/MyDrive/mal_corpus_minV2.txt'
text = open(path).read()
print('length of the corpus is: :', len(text))

length of the corpus is: : 303234


In [11]:

#tokeniziting..
tokenizer = RegexpTokenizer(r'\^w+|$[0-9]+|\S+')
words = tokenizer.tokenize(text)


In [12]:
print(len(words))

31360


In [13]:
#getting unique words..
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [14]:

print(len(unique_words))

16093


In [15]:
print(unique_words[2700:2705])

['ഉപയോഗം,' 'ഉപയോഗകാലം' 'ഉപയോഗക്ഷമതയെ' 'ഉപയോഗക്ഷമമാക്കാനുള്ള'
 'ഉപയോഗയോഗ്യമല്ല.']


In [44]:

WORD_LENGTH = 5
next_words = []
prev_words = []
for j in range(len(words) - WORD_LENGTH):
     prev_words.append(words[j:j + WORD_LENGTH])
     next_words.append(words[j + WORD_LENGTH])
print(prev_words[1300])
print(next_words[1300])

['സ്ഥാപിതമായി.', 'മേരിലാന്റ്', 'ബാൽട്ടിമോറിലെ', 'ഹോംവുഡ്', 'കാമ്പസിൽ']
സ്ഥിതിചെയ്യുന്ന


In [77]:
print(prev_words[2508])
print(next_words[2508])

['ഏഴു', 'പേർക്ക്', 'നൂറിലധികം', 'പ്രത്യേക', 'ഉപകരണങ്ങൾ']
ഉപയോഗിച്ച്


In [136]:
#X features.. Y feaures
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1


In [137]:
#LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))


In [138]:
#Model training...
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=10, shuffle=True).history


Epoch 1/10
233/233 [==============================] - 14s 50ms/step - loss: 8.5253 - accuracy: 0.1123 - val_loss: 9.5408 - val_accuracy: 0.0351
Epoch 2/10
233/233 [==============================] - 11s 47ms/step - loss: 7.9559 - accuracy: 0.1307 - val_loss: 9.7248 - val_accuracy: 0.0364
Epoch 3/10
233/233 [==============================] - 11s 47ms/step - loss: 7.6360 - accuracy: 0.1449 - val_loss: 10.1465 - val_accuracy: 0.0364
Epoch 4/10
233/233 [==============================] - 11s 47ms/step - loss: 7.4217 - accuracy: 0.1736 - val_loss: 10.4715 - val_accuracy: 0.0370
Epoch 5/10
233/233 [==============================] - 11s 47ms/step - loss: 7.2799 - accuracy: 0.2140 - val_loss: 10.8535 - val_accuracy: 0.0370
Epoch 6/10
233/233 [==============================] - 11s 47ms/step - loss: 7.1814 - accuracy: 0.2470 - val_loss: 11.1295 - val_accuracy: 0.0357
Epoch 7/10
233/233 [==============================] - 11s 47ms/step - loss: 7.1165 - accuracy: 0.2729 - val_loss: 11.3571 - val_accu

In [153]:
# model.save('keras_next_word_model.h5')

In [154]:
# pickle.dump(history, open("history.p", "wb"))

In [46]:
model = load_model('/content/drive/MyDrive/keras_next_word_model.h5')

In [157]:
history = pickle.load(open("history.p", "rb"))

In [166]:
# #Model Evaluation..
# plt.plot(history['accuracy'])
# plt.plot(history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')


In [167]:

# plt.plot(history['loss'])
# plt.plot(history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')

In [47]:
def prepare_input(text):
    
    x = np.zeros((1, len(text.split(" ")), len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x


In [78]:
str_= "ഏഴു പേർക്ക് ഉപകരണങ്ങൾ ഉപയോഗം"

In [79]:
prepare_input(str_)

ഏഴു
പേർക്ക്
ഉപകരണങ്ങൾ
ഉപയോഗം


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [50]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [51]:
def predict_completions(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]

In [52]:
# quotes = [
#     "ആദ്യമായി ഒളിമ്പിക്സിൽ വ്യക്തഗത ഇനത്തിൽ ഇന്ത്യക്കു വേണ്ടി മെഡൽ നേടിയ കായിക താരമെന്ന ബഹുമതിയും ജാദവിനുള്ളതാണ്.",
#     "ഫിൻലൻഡിലെ ഹെൽസിങ്കിയിൽ വച്ചു നടന്ന വേനൽക്കാല ഒളിമ്പിക്സിൽ പുരുഷന്മാരുടെ ഗുസ്തി മത്സരത്തിൽ ഇദ്ദേഹത്തിനു വെങ്കല മെഡൽ ലഭിച്ചു.",
#     "ഝാർഖണ്ഡിലുള്ള ജാംഷെഡ്‌പൂർ എന്ന സ്ഥലത്താണ് തനുശ്രീ ദത്ത ജനിച്ചതും വളർന്നതും."

# ]

In [64]:
# for q in quotes:
#     seq = q[:40].lower()
#     print(seq)
#     print(predict_completions(seq,3))
#     print()




In [81]:
# print(predict_completions(seq,3))